## Preamble

In [ ]:
# Required packages
import numpy as np
import cv2 
from scipy.signal import savgol_filter
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib as mpl
from math import *
import tifffile
import warnings
import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.patches as mpatches
from scipy.optimize import curve_fit
import string
from pylab import *
from PIL import Image
from osgeo import gdal as GD  
warnings.filterwarnings("ignore")

In [ ]:
# Plot formatting 
matplotlib.rcParams['mathtext.fontset']='cm'
matplotlib.rcParams['font.family']='STIXGeneral'
plt.rcParams['legend.fontsize']=15
plt.rcParams.update({'font.size':15}) 
plt.rcParams['axes.axisbelow']=True

# Read the reconstructed micro-tomographic slices

In [ ]:
### IMPORT RAW IMAGES ###

# path to the images 
path = "38 images/slice119_crop_bottom_brightness20000-54067_cropped-for-Luke-segmentation_00"

# make a list that contains the name of the images padded with zeroes
select = list(range(0,37,1))
for i in range(37):
    select[i]='%0.2d' % i
    
# read image, flatten and append to "lists"
lists = []
for img in select:
    n = cv2.imread(str(path)+img+".tif")
    data = n.flatten()
    lists.append(data)


# Find the thresholds in the residual


### 1. Fit a Gaussian

In [ ]:
def gauss(x,mu,sigma,A):
        return A*exp(-(x-mu)**2/2/sigma**2)

In [ ]:
### Fit Gaussian to Image #35 ###
fig, ax = plt.subplots(1,1, figsize=(10,5))
i=35 #image number 35 

#fit Gaussian
x0, sigma = 0, 0.1
bins = np.arange(0,255,1)
y_tot, xe  = np.histogram(lists[i], bins)
x = .5 * (xe[:-1] + xe[1:])
p0 = [1., 1., 1.]
fit, tmp = curve_fit(gauss, x, y_tot, p0=p0)

###################### PLOT #######################
ax.plot(x, y_tot, c='dodgerblue', label='Grey Scale Data', zorder=0, lw = 3)
x_fine = np.linspace(xe[0], xe[-1], 254)
y_gauss_tot = gauss(x_fine, fit[0], fit[1], fit[2])
ax.plot(x_fine, gauss(x_fine, fit[0], fit[1], fit[2]), c='darkorange',ls='--', lw=5 ,label='Best-fit Gaussian', zorder=4)
ax.set_xlabel('Grey Values')
ax.set_ylabel('Frequency')
ax.legend()
ax.grid()

### 2. Calculate Residual and find the minimum and the inflection points

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,5))

## RESIDUAL 
diff_abs = np.abs(y_gauss_tot - y_tot)

## MINIMUM 
fder = np.gradient(diff_abs) #first derivative
inds = np.where(np.diff(np.sign(fder))) #find change in sign
inds=list(chain.from_iterable(inds)) # indices of all sign changes
min_val = []
for n in inds:
    val=diff_abs[n]
    min_val.append(val)
D=pd.DataFrame({'inds':inds,'min_val':min_val})
# focus search of sign change to band of minimum values between 100 and 950
D = D.drop(D[D.min_val > 950].index) 
D = D.drop(D[D.inds < 100].index)
D=D.reset_index(drop=True)
UP_threshold = D.inds[0] #upper threshold 

## INFLECTION POINT
yhat = savgol_filter(diff_abs, 61, 3) # window size , polynomial order 
diff_1= np.gradient(yhat) # 2nd derivative
diff_2= np.gradient(diff_1) # 3rd derivative
indices = np.where(np.diff(np.sign(diff_2))) # Find the inflection point.
indices=list(chain.from_iterable(indices))
inflect_val=[]
for a in indices: 
    val = diff_abs[a]
    inflect_val.append(val)
I=pd.DataFrame({'indices':indices,'inflect_val':inflect_val})
# focus the smoothing between the indices 50 and 80 of the greyscale
I = I.drop(I[I.indices < 50].index)
I = I.drop(I[I.indices > 80].index)
I=I.reset_index(drop=True)
x_fine_ = x_fine[0:140] #x-axis
LOW_threshold = I['indices'].iloc[-1] #lower threshold

###################### PLOT #######################
ax.plot(x_fine, diff_abs, c='dodgerblue', label='Residual', lw = 3) #actual residual
ax.plot(x_fine, yhat, c='darkorange', ls='--', lw=5, label='Smoothed Residual') #smoothed residual
ax.axvline(x=UP_threshold,c='black',lw=3) #plotting the threshold on the residual plot
ax.axvline(x=LOW_threshold,c='black', ls = '-.', lw=3)
ax.set_xlabel('Grey Values')
ax.set_ylabel('Absolute Difference')
ax.grid()
ax.legend()
plt.show()